In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [3]:
os.getenv("GROQ_API_KEY")

'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'

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [5]:
response = llm.invoke("What it the capital of India")

In [8]:
print(response.content)

The capital of India is actually two cities: New Delhi serves as the country's capital for legislative purposes, while Mumbai (formerly known as Bombay) is the financial and entertainment capital.

New Delhi, located in the northern part of the country, is home to the Indian government and its various institutions, including the Parliament of India and the Rashtrapati Bhavan, which serves as the official residence of the President of India. The city is known for its rich history, diverse culture, and numerous tourist attractions, such as the Red Fort, India Gate, and the Lotus Temple.

Mumbai, located on the west coast of India, is the country's largest city and a major hub for finance, commerce, and entertainment. It is home to the Bombay Stock Exchange, as well as the Indian film industry, known as Bollywood. The city is known for its vibrant energy, diverse population, and iconic landmarks, such as the Gateway of India and the Chhatrapati Shivaji Maharaj Vastu Sangrahalaya (formerly

In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/136260588966683334-software-engineer-ii-full-stack-youtube")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [16]:
docs = loader.load().pop().page_content

In [17]:
docs

"Software Engineer II, Full Stack, YouTube — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend Feedbackjob detailsarrow_backBack to jobs searchJobs search results2,497  jobs matchedSoftware Engineer II, Full Stack, YouTubeZürich, SwitzerlandContact Center AI Sales Specialist IV, Google CloudAustin, TX, USA; Cambridge, MA, USA; +6 more; +5 moreReverse and Returns Logistics ManagerLas Vegas, Nevada, USA; Papillion, NE, USA; +5 more; +4 moreSenior Engineering Manager, Search and ComplianceBengaluru, Karnataka, IndiaSenior UX Researcher, gTech Users and ProductsBoulder, CO, USA; Atlanta, GA, USACloud Consultant, Application Modernization Edge, Professional ServicesAustin, TX, USA; Atlanta, GA, USA; +2 more; +1 moreCamera Software Engineer, Project StarlineMountain View, CA, USA;

In [26]:
from langchain_core.prompts import PromptTemplate

In [29]:
extraction_prompt = PromptTemplate.from_template(
    '''
    ### SCRAPED TEXT FROM WEBSITE
    {page_content}
    ### INSTRUCTION
    Here is the page response from which you have to extract the job role, experience, skills, and description required for this job in JSON format. 
    Make sure to only extract the information specifically requested, and there should be NO PREAMBLE.
    The keys in the output should be 'job role', 'experience', 'skills', and 'description'.
    ### RETURN VALID OUTPUT IN JSON FORMAT
    '''
)


In [30]:
chain_extract = extraction_prompt | llm 

In [38]:
response = chain_extract.invoke(input={'page_content':docs})

In [45]:
response.content

'{\n  "job role": "Software Engineer II, Full Stack",\n  "experience": "Minimum qualifications: Bachelor’s degree or equivalent practical experience. 1 year of experience with software development in one or more programming languages (e.g., Python, C, C++, Java, JavaScript). 1 year of experience with data structures or algorithms. 1 year of experience with full stack development, across back-end such as Java, Python, GO, or C++ codebases, and front-end experience including JavaScript or TypeScript, HTML, CSS or equivalent.",\n  "skills": "Preferred qualifications: Experience in C++ programming language. Experience in understanding ML systems. Experience in e-commerce software. Familiarity with YouTube or Google Commerce tech stack. Excellent communication skills for collaborating with both technical and non-technical team members.",\n  "description": "Google\'s software engineers develop the next-generation technologies that change how billions of users connect, explore, and interact w

In [46]:
from langchain_core.output_parsers import JsonOutputParser

In [47]:
parser = JsonOutputParser()
json_response = parser.parse(response.content)

In [48]:
type(json_response)

dict

In [52]:
json_response

{'job role': 'Software Engineer II, Full Stack',
 'experience': 'Minimum qualifications: Bachelor’s degree or equivalent practical experience. 1 year of experience with software development in one or more programming languages (e.g., Python, C, C++, Java, JavaScript). 1 year of experience with data structures or algorithms. 1 year of experience with full stack development, across back-end such as Java, Python, GO, or C++ codebases, and front-end experience including JavaScript or TypeScript, HTML, CSS or equivalent.',
 'skills': 'Preferred qualifications: Experience in C++ programming language. Experience in understanding ML systems. Experience in e-commerce software. Familiarity with YouTube or Google Commerce tech stack. Excellent communication skills for collaborating with both technical and non-technical team members.',
 'description': "Google's software engineers develop the next-generation technologies that change how billions of users connect, explore, and interact with informat

In [53]:
cold_email_tempelate = PromptTemplate.from_template(
    '''
        ### JOB DESCRIPTION
        {job_description}

        ### INSTRUCTION
        My name is {name} and I have gained knowledge in {skills} and want to apply for {job_role} which required {required_skills} these skills. 
        your job is to Generate a professional and personalized cold email to the hiring manager for the job described above and make sure you mentions the the necessary skills from my 
        skillset which is required for this job role. The email should convey enthusiasm, professionalism, and the candidate's desire to contribute to the 
        company’s mission. The tone should be respectful and focused on how the candidate can add value to the team.
        Also, include a call to action for scheduling an interview or further discussion. The email should be around 150-200 words.

        ### NO PREAMBLE

    '''
)

In [54]:
email_chain = cold_email_tempelate | llm

In [55]:
name = "mohit Dwivedi"
skills= ['Python','Machine Learning','Deep learning','AI and ML','Mongo DB','React Native','Full Stack']

In [58]:
job_role = json_response['job role']
experience_required = json_response['experience']
skills_required = json_response['skills']
job_description = json_response['description']

In [ ]:
email_chain.invoke(input = {'job_description':job_description,'name': name, 'skills':skills,'required_skills':skills_required})